#### Code to validate models

In [362]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [363]:
cd ../src

/home/theo/kaggle/siim_covid/src


## Imports

In [364]:
import os
import cv2
import json
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import *
from collections import Counter
from tqdm.notebook import tqdm

In [365]:
from params import *

In [366]:
from data.preparation import prepare_dataframe, handle_duplicates
from data.dataset import CovidDetDataset, CovidClsDataset
from data.transforms import get_transfos_det, get_transfos_cls

from model_zoo.models import get_model

from utils.plot import plot_sample

from utils.logger import Config

from utils.metrics import per_class_average_precision_score, study_level_map

## Data

### Load

In [367]:
EXP_FOLDERS = [
    LOG_PATH + "2021-07-30/4/",
    LOG_PATH + "2021-07-31/0/",
#     LOG_PATH + "2021-08-01/0/",
#     LOG_PATH + "2021-08-01/1/",
#     LOG_PATH +  "aphrodeep_v2s_lung/"
]

EXP_FOLDER = EXP_FOLDERS[-1]

USE_TTA = True

In [368]:
config = Config(json.load(open(EXP_FOLDER + "config.json", 'r')))

In [369]:
if USE_TTA:
    pred_oof_img = np.mean([np.load(f + "pred_oof_img_flip.npy") for f in EXP_FOLDERS], 0)
    pred_oof_study = np.mean([np.load(f + "pred_oof_study_flip.npy") for f in EXP_FOLDERS], 0)
else:
    pred_oof_img = np.mean([np.load(f + "pred_oof_img.npy") for f in EXP_FOLDERS], 0)
    pred_oof_study = np.mean([np.load(f + "pred_oof_study.npy") for f in EXP_FOLDERS], 0)

In [370]:
df = pd.read_csv(EXP_FOLDER + "data.csv")

pred_cols = [c + "_pred" for c in CLASSES]
df[pred_cols] = pred_oof_study
df['pred_img'] = pred_oof_img

In [371]:
# for i in range(5):
#     print(df[df['kfold'] == i][pred_cols].max(0))

In [372]:
ap = per_class_average_precision_score(pred_oof_img, df['img_target'].values)
print(f'Image mAP : {ap :.4f}')

auc = roc_auc_score(df['img_target'], pred_oof_img)
print(f'Image AUC : {auc :.4f}')
acc = accuracy_score(df['img_target'], pred_oof_img > 0.5)
print(f'Image Acc : {acc :.4f}')

Image mAP : 0.9604
Image AUC : 0.9108
Image Acc : 0.8465


In [373]:
study_map = study_level_map(df[pred_cols].values, df[CLASSES].values, df['study_id'].values)
print(f'Study mAP : {study_map :.4f}')

Study mAP : 0.3831


### Post-process

#### Img merging

In [374]:
groups = df[['study_id', 'pred_img']].groupby('study_id').mean().rename(
    columns={'pred_img': 'pred_img_merged'}
).reset_index()
df_ = df.merge(groups, on="study_id", how="left")

# df_.loc[df_['negative_pred'] > 0.75, 'pred_img'] = 0
# df_.loc[df_['typical_pred'] > 0.75, 'pred_img'] = 1
# df_.loc[df_['indeterminate_pred'] > 0.75, 'pred_img'] = 1

In [375]:
ap = per_class_average_precision_score(df_['pred_img_merged'].values, df['img_target'].values)
print(f'Image mAP : {ap :.4f}')
auc = roc_auc_score(df_['img_target'], df_['pred_img_merged'])
print(f'Image AUC : {auc :.4f}')
acc = accuracy_score(df_['img_target'], df_['pred_img_merged'] > 0.5)
print(f'Image Acc : {acc :.4f}')

Image mAP : 0.9605
Image AUC : 0.9110
Image Acc : 0.8470


#### Study using img

In [376]:
df_ = df.copy()

p = 0.5
df_['negative_pred'] *= (1 - df_['pred_img']) ** p
df_['typical_pred'] *= (df_['pred_img']) ** p

# df_['indeterminate_pred'] *= (df_['pred_img']) ** p
# df_['atypical_pred'] *= (df_['pred_img']) ** p

study_map = study_level_map(df_[pred_cols].values, df_[CLASSES].values, df['study_id'].values)
print(f'Study mAP : {study_map :.5f}')

Study mAP : 0.38327


In [377]:
df_study = df[
    ['study_id'] + pred_cols + CLASSES + ['img_target', 'pred_img']
].groupby('study_id').agg(np.mean).copy()

# df_study['negative_pred'] *= 1 - df_study['pred_img'] 
# df_study['typical_pred'] *= df_study['pred_img'] 
# df_study['indeterminate_pred'] *= df_study['pred_img'] 
# df_study['atypical_pred'] *= df_study['pred_img'] 

df_study.loc[df_study['pred_img'] > 0.75, 'negative_pred'] *= 0.5
df_study.loc[df_study['pred_img'] < 0.2, 'negative_pred'] *= 2

df_study.loc[df_study['pred_img'] > 0.75, 'typical_pred'] *= 1.2
# df_study.loc[df_study['pred_img'] > 0.9, 'indeterminate_pred'] *= 1.1
# df_study.loc[df_study['pred_img'] > 0.9, 'atypical_pred'] *= 1.1

df_study.loc[df_study['pred_img'] < 0.25, 'typical_pred'] *= 0.8
# df_study.loc[df_study['pred_img'] < 0.2, 'indeterminate_pred'] *= 0.9
# df_study.loc[df_study['pred_img'] < 0.2, 'atypical_pred'] *= 0.9

accs = per_class_average_precision_score(
    df_study[pred_cols].values,
    df_study[CLASSES].values, 
    num_classes=NUM_CLASSES, 
    average=False
)
np.round(accs, 4), np.round(np.mean(accs) * 2/3, 4)

(array([0.802 , 0.8478, 0.3069, 0.3421]), 0.3831)

## Results

In [378]:
def proc(x):
    x = x.split('0 0 1 1')[:4]
    x = [float(y.strip().split(' ')[1]) for y in x]
    return x

In [379]:
sub = pd.read_csv("../output/sub_0931.csv")

In [380]:
sub = pd.read_csv("../output/sub_0931.csv")
sub_study = sub[sub['id'].apply(lambda x: "study" in x)].copy()

pred_test = np.array(sub_study['PredictionString'].apply(proc).values.tolist())

for i, c in enumerate(CLASSES):
    sub_study[c] = pred_test[:, i]
    
sub_study.to_csv("../output/sub_0931_study.csv", index=False)

In [381]:
sub = pd.read_csv("../output/sub_0931.csv")

sub_img = sub[sub['id'].apply(lambda x: "study" not in x)].copy()
sub_img['none'] = sub_img['PredictionString'].apply(lambda x: float(x.split('none')[-1].strip().split(' ')[0]))

sub_img.to_csv("../output/sub_0931_img.csv", index=False)

In [382]:
def treat_oof(pred_oof_study):
    pred_oof_study['id'] = pred_oof_study['id'].apply(lambda x: x.split('_')[0])
    
    df_study = pd.read_csv(DATA_PATH + "train_study_level.csv")
    df_study['study_id'] = df_study['id'].apply(lambda x: x.split('_')[0])
    df_study = df_study.rename(columns={c: c.split(' ')[0].lower() for c in df_study.columns})

    df_study.drop('id', axis=1, inplace=True)

    df_study = df_study.merge(pred_oof_study, how="left", left_on="study_id", right_on="id").dropna()
    
    pred_oof_study = np.array(df_study['PredictionString'].apply(proc).values.tolist())
    
    df_study[pred_cols] = pred_oof_study
    
    df_g = df[['study_id'] + pred_cols].groupby('study_id').mean().reset_index()
    df_study = df_study.merge(df_g, how="left", left_on="id", right_on="study_id", suffixes=['', '_theo'])
    
    return df_study

In [383]:
df_old = treat_oof(pd.read_csv('../output/OOF_study_only_EBV2M_768.csv'))
df_v2m_2 = treat_oof(pd.read_csv('../output/oof_v2m.csv'))
df_b4 = treat_oof(pd.read_csv('../output/oof_b4.csv'))
df_b5 = treat_oof(pd.read_csv('../output/oof_b5.csv'))
df_ono = treat_oof(pd.read_csv('../output/oof_ono.csv')[['id', 'PredictionString']])
df_v2m_640 = treat_oof(pd.read_csv('../output/submit_OOF_V2M_640.csv'))
df_b6 = treat_oof(pd.read_csv('../output/submit_OOF_B6_512.csv'))

In [384]:
df_theo = df[['study_id'] + pred_cols + CLASSES].groupby('study_id').mean().reset_index()
df_theo = df_v2m_2[['id']].merge(df_theo, how="left", left_on="id", right_on="study_id")

df_ono = df_v2m_2[['id']].merge(df_ono, how="left", left_on="id", right_on="study_id")

In [385]:
p = 1

oofs = [
#     df_old[pred_cols].values ** p * 1,
    df_theo[pred_cols].values ** p * 1,
    df_v2m_2[pred_cols].values ** p * 1,
#     df_b4[pred_cols].values ** p * 0.1,
    df_b5[pred_cols].values ** p * 0.5,
#     df_ono[pred_cols].values ** p * 0.5,
    df_b6[pred_cols].values ** p * 0.5,
#     df_v2m_640[pred_cols].values ** p * 1,
]

oof = np.mean(oofs, 0)

In [386]:
from scipy.stats import pearsonr

In [387]:
truth = df_theo[CLASSES].values
pred = oof
aucs = [roc_auc_score(truth[:, i].flatten(), pred[:, i].flatten()) for i in range(pred.shape[1])]
aucs

[0.9243240146654443,
 0.8716498208065167,
 0.6946356202443238,
 0.8080906189641159]

In [388]:
per_class_average_precision_score(oof, df_theo[CLASSES].values, num_classes=NUM_CLASSES) * 2 / 3

0.39228991794419105

In [389]:
per_class_average_precision_score(oof, df_theo[CLASSES].values, num_classes=NUM_CLASSES, average=False) #* 2 / 3

array([0.81232886, 0.85242912, 0.32088746, 0.36809407])

## Blend img

In [390]:
oof_mk = pd.read_csv("../output/oof_image_level_tf_efficientnet_b4_ns.csv")
oof_mk = oof_mk.merge(df, on='image_id', how='left')[['image_id', 'img_target', 'pred_img', 'pred']].dropna()

In [391]:
oof_mk_2 = pd.read_csv("../output/oof_image_level_tf_efficientnet_b3_ns.csv")
oof_mk_2 = oof_mk_2.merge(df, on='image_id', how='left')[['image_id', 'img_target', 'pred_img', 'pred']].dropna()

In [392]:
oof_mk_3 = pd.read_csv("../output/oof_image_level_tf_efficientnet_b5_ns.csv")
oof_mk_3 = oof_mk_3.merge(df, on='image_id', how='left')[['image_id', 'img_target', 'pred_img', 'pred']].dropna()

In [393]:
file = oof_mk_2
ap = per_class_average_precision_score(file['pred'].values, file['img_target'].values)
print(f'Image mAP : {ap :.4f}')
auc = roc_auc_score(file['img_target'], file['pred'])
print(f'Image AUC : {auc :.4f}')
# acc = accuracy_score(file['pred'] > 0.5, file['img_target'] > 0.5)
# print(f'Image Acc : {acc :.4f}')

Image mAP : 0.9531
Image AUC : 0.8940


In [394]:
ap = per_class_average_precision_score(oof_mk['pred_img'].values, oof_mk['img_target'].values)
print(f'Image mAP : {ap :.4f}')
auc = roc_auc_score(oof_mk['img_target'], oof_mk['pred_img'])
print(f'Image AUC : {auc :.4f}')
acc = accuracy_score(oof_mk['pred_img'] > 0.5, oof_mk['img_target'] > 0.5)
print(f'Image Acc : {acc :.4f}')

Image mAP : 0.9604
Image AUC : 0.9109
Image Acc : 0.8460


In [417]:
P = 1
oof_mk['blend'] = oof_mk['pred_img'] ** P * 3 + oof_mk_2['pred'] ** P * 0.5 + oof_mk_3['pred'] ** P * 1 + oof_mk['pred'] ** P * 0
oof_mk['blend']  = oof_mk['blend'] / oof_mk['blend'].max()

In [418]:
ap = per_class_average_precision_score(oof_mk['blend'].values, oof_mk['img_target'].values)
print(f'Image mAP : {ap :.4f}')
auc = roc_auc_score(oof_mk['img_target'], oof_mk['blend'])
print(f'Image AUC : {auc :.4f}')

Image mAP : 0.9626
Image AUC : 0.9127


# Final blend

In [326]:
oof_mk["blend_mk"] = (oof_mk_2['pred'] + oof_mk_3['pred'] + oof_mk['pred']) / 3

In [327]:
oof_mk = oof_mk.merge(df[['study_id', 'image_id']] , how="left", on="image_id")

In [328]:
oof_mk = oof_mk.merge(df_theo, how="left", on="study_id")

In [329]:
oof_mk = oof_mk.dropna()

In [330]:
oof_mk.to_csv('../output/final_blend.csv', index=False)

In [332]:
study_level_map(oof_mk[pred_cols].values, oof_mk[CLASSES].values, oof_mk['study_id'].values)

0.38319520568661075

In [333]:
per_class_average_precision_score(oof_mk['blend'].values, oof_mk['img_target'].values)

0.9624656477013465

In [334]:
df = oof_mk.copy()
df['pred_img'] = df['blend']

In [359]:
df_ = df.copy()

p = 1
df_['negative_pred'] *= (1 - df_['blend_mk']) ** p
df_['typical_pred'] *= (df_['blend']) ** p

# df_['negative_pred'] -= df_['blend_mk'] ** p
# df_['typical_pred'] += df_['blend'] ** p

# df_['indeterminate_pred'] *= (df_['pred_img']) ** p
# df_['atypical_pred'] += (df_['blend_mk']) ** p

study_map = study_level_map(df_[pred_cols].values, df_[CLASSES].values, df['study_id'].values)
print(f'Study mAP : {study_map :.5f}')

Study mAP : 0.38391


In [207]:
df_study = df[
    ['study_id'] + pred_cols + CLASSES + ['img_target', 'pred_img']
].groupby('study_id').agg(np.mean).copy()

# df_study['negative_pred'] *= 1 - df_study['pred_img'] 
# df_study['typical_pred'] *= df_study['pred_img'] 
# df_study['indeterminate_pred'] *= df_study['pred_img'] 
# df_study['atypical_pred'] *= df_study['pred_img'] 

df_study.loc[df_study['pred_img'] > 0.75, 'negative_pred'] *= 0.5
df_study.loc[df_study['pred_img'] < 0.2, 'negative_pred'] *= 2

df_study.loc[df_study['pred_img'] > 0.75, 'typical_pred'] *= 1.2
# df_study.loc[df_study['pred_img'] > 0.9, 'indeterminate_pred'] *= 1.1
# df_study.loc[df_study['pred_img'] > 0.9, 'atypical_pred'] *= 1.1

df_study.loc[df_study['pred_img'] < 0.25, 'typical_pred'] *= 0.8
# df_study.loc[df_study['pred_img'] < 0.2, 'indeterminate_pred'] *= 0.9
# df_study.loc[df_study['pred_img'] < 0.2, 'atypical_pred'] *= 0.9

accs = per_class_average_precision_score(
    df_study[pred_cols].values,
    df_study[CLASSES].values, 
    num_classes=NUM_CLASSES, 
    average=False
)
np.round(accs, 4), np.round(np.mean(accs) * 2/3, 4)

(array([0.8023, 0.8477, 0.3076, 0.3423]), 0.3833)